# Food Donation Nearest Neighbor Matching Algorithm

This notebook implements an AI-based nearest neighbor matching system for food donations.
**Linux-compatible version** for running on cloud instances.

## Ranking Strategies:
1. **Balanced Priority** (Default): Distance (40%) + Expiry urgency (60%)
2. **Distance First**: Prioritizes nearest donations
3. **Urgency First**: Prioritizes expiring-soon donations

## Features:
- ✅ Linux/Unix compatible (no OS-specific dependencies)
- ✅ Connects to Supabase database
- ✅ Calculates great-circle distance using Haversine formula
- ✅ Geocodes addresses when GPS coordinates are missing
- ✅ Multiple sorting strategies
- ✅ Filters out infeasible deliveries (can't reach before expiry)
- ✅ Visualizations and CSV export

In [ ]:
# Install required packages (Linux-compatible)
import sys
print(f"Python version: {sys.version}")
print(f"Platform: {sys.platform}")

# Install packages
!pip install --quiet supabase pandas geopy requests python-dotenv matplotlib

print("✅ All packages installed successfully!")

In [ ]:
# Import required libraries
import os
import sys
import math
import pandas as pd
from datetime import datetime, timezone
from geopy.distance import great_circle
from supabase import create_client, Client
import requests
from dotenv import load_dotenv
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Load environment variables from multiple possible locations
env_paths = ['.env', '../.env', '../../.env', '/app/.env']
for env_path in env_paths:
    if os.path.exists(env_path):
        load_dotenv(env_path)
        print(f"✅ Loaded environment from: {env_path}")
        break
else:
    print("⚠️ No .env file found in standard locations")

print("✅ All libraries imported successfully!")
print(f"Running on: {sys.platform}")

In [ ]:
# Configure Supabase connection
# Replace with your actual Supabase credentials
SUPABASE_URL = os.getenv('VITE_SUPABASE_URL', 'YOUR_SUPABASE_URL')
SUPABASE_KEY = os.getenv('VITE_SUPABASE_ANON_KEY', 'YOUR_SUPABASE_ANON_KEY')

# Create Supabase client
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

print("✅ Connected to Supabase!")
print(f"URL: {SUPABASE_URL[:30]}...")

## 1. Helper Functions

In [ ]:
def calculate_distance(lat1, lon1, lat2, lon2):
    """
    Calculate great circle distance between two points using Haversine formula
    Returns distance in kilometers
    """
    # Using geopy for accuracy
    coords_1 = (lat1, lon1)
    coords_2 = (lat2, lon2)
    distance_km = great_circle(coords_1, coords_2).km
    return distance_km


def calculate_urgency_score(expiry_datetime):
    """
    Calculate urgency score based on time until expiry
    Returns score 0-100 (higher = more urgent)
    """
    now = datetime.now(timezone.utc)
    
    # Handle string datetime
    if isinstance(expiry_datetime, str):
        expiry = datetime.fromisoformat(expiry_datetime.replace('Z', '+00:00'))
    else:
        expiry = expiry_datetime
    
    time_until_expiry_minutes = (expiry - now).total_seconds() / 60
    
    if time_until_expiry_minutes <= 0:
        return 100  # Already expired
    elif time_until_expiry_minutes <= 60:
        return 90   # Less than 1 hour
    elif time_until_expiry_minutes <= 180:
        return 70   # Less than 3 hours
    elif time_until_expiry_minutes <= 360:
        return 50   # Less than 6 hours
    elif time_until_expiry_minutes <= 720:
        return 30   # Less than 12 hours
    elif time_until_expiry_minutes <= 1440:
        return 10   # Less than 24 hours
    else:
        return 0    # More than 24 hours


def calculate_feasibility(distance_km, expiry_datetime):
    """
    Calculate if delivery is feasible (can reach before expiry)
    Average speed: 20 km/h
    """
    AVERAGE_SPEED_KMH = 20  # kilometers per hour
    
    # Calculate travel time
    travel_time_hours = distance_km / AVERAGE_SPEED_KMH
    travel_time_minutes = math.ceil(travel_time_hours * 60)
    
    # Calculate time until expiry
    now = datetime.now(timezone.utc)
    if isinstance(expiry_datetime, str):
        expiry = datetime.fromisoformat(expiry_datetime.replace('Z', '+00:00'))
    else:
        expiry = expiry_datetime
    
    time_until_expiry_minutes = (expiry - now).total_seconds() / 60
    
    return {
        'distance_km': round(distance_km, 2),
        'travel_time_minutes': travel_time_minutes,
        'time_until_expiry_minutes': int(time_until_expiry_minutes),
        'is_feasible': travel_time_minutes < time_until_expiry_minutes
    }


def geocode_address(donation):
    """
    Geocode address to get coordinates using OpenStreetMap Nominatim
    Returns (lat, lng) tuple or None
    """
    address = f"{donation['pickup_street_address']}, {donation.get('pickup_area', '')}, {donation['pickup_city']}, {donation['pickup_state']}, {donation['pickup_pin_code']}"
    
    try:
        geocode_url = f"https://nominatim.openstreetmap.org/search?format=json&q={requests.utils.quote(address)}"
        response = requests.get(geocode_url, headers={'User-Agent': 'FoodBridge/1.0'})
        data = response.json()
        
        if data and len(data) > 0:
            lat = float(data[0]['lat'])
            lng = float(data[0]['lon'])
            
            # Update database with geocoded coordinates
            supabase.table('donations').update({
                'pickup_latitude': lat,
                'pickup_longitude': lng
            }).eq('id', donation['id']).execute()
            
            return (lat, lng)
    except Exception as e:
        print(f"⚠️ Geocoding error: {e}")
    
    return None

print("✅ Helper functions defined!")

## 2. Set Recipient Location

Enter the recipient's coordinates (or use a default location)

## 2A. Choose Sorting Strategy

Select how you want donations to be ranked:
- **'balanced'**: Default - Combines distance (40%) and urgency (60%)
- **'distance'**: Nearest first - Prioritizes closest donations
- **'urgency'**: Expiring first - Prioritizes donations expiring soonest

In [ ]:
# SORTING STRATEGY CONFIGURATION
# Options: 'balanced', 'distance', 'urgency'

SORTING_STRATEGY = 'distance'  # ← Change this to your preferred strategy

print(f"🎯 Selected Sorting Strategy: {SORTING_STRATEGY.upper()}")
print()
if SORTING_STRATEGY == 'balanced':
    print("   📊 Balanced approach: Distance (40%) + Urgency (60%)")
    print("   Best for: General use, balanced recommendations")
elif SORTING_STRATEGY == 'distance':
    print("   📍 Distance-first: Nearest donations ranked highest")
    print("   Best for: Quick pickups, minimizing travel time")
elif SORTING_STRATEGY == 'urgency':
    print("   ⏰ Urgency-first: Expiring-soon donations ranked highest")
    print("   Best for: Preventing food waste, handling critical donations")
else:
    print(f"   ⚠️ Unknown strategy '{SORTING_STRATEGY}', defaulting to 'balanced'")
    SORTING_STRATEGY = 'balanced'

In [ ]:
# Set recipient location (example: Bangalore, India)
RECIPIENT_LOCATION = {
    'lat': 12.9716,   # Latitude
    'lng': 77.5946    # Longitude
}

print(f"📍 Recipient Location: {RECIPIENT_LOCATION['lat']}, {RECIPIENT_LOCATION['lng']}")
print("   (Change these values to your actual location)")

## 3. Fetch All Available Donations from Supabase

In [ ]:
# Fetch all donations with status='successful'
response = supabase.table('donations').select('*').eq('status', 'successful').order('created_at', desc=True).execute()

donations = response.data

print(f"📊 Found {len(donations)} available donations")

if len(donations) > 0:
    # Display basic info
    df = pd.DataFrame(donations)
    display_cols = ['id', 'food_name', 'food_type', 'quantity', 'unit', 'pickup_city', 'expiry_datetime']
    print("\n📋 Available Donations:")
    display(df[display_cols].head(10))
else:
    print("⚠️ No donations found with status='successful'")

## 4. Run Nearest Neighbor Optimization Algorithm

In [ ]:
def find_optimal_donations(recipient_location, donations, strategy='balanced'):
    """
    Nearest Neighbor Matching Algorithm with multiple sorting strategies
    
    Args:
        recipient_location: dict with 'lat' and 'lng' keys
        donations: list of donation dictionaries
        strategy: 'balanced', 'distance', or 'urgency'
    
    Returns:
        DataFrame with ranked donations
    """
    print("🤖 Starting AI Optimization...")
    print(f"   Strategy: {strategy.upper()}")
    print("="*60)
    
    results = []
    
    for donation in donations:
        donor_lat = donation.get('pickup_latitude')
        donor_lng = donation.get('pickup_longitude')
        
        # If coordinates missing, try geocoding
        if not donor_lat or not donor_lng:
            print(f"\n⚠️ Donation #{donation['id']}: Missing coordinates, attempting geocode...")
            coords = geocode_address(donation)
            
            if coords:
                donor_lat, donor_lng = coords
                print(f"   ✅ Geocoded: {donor_lat}, {donor_lng}")
            else:
                print(f"   ❌ Geocoding failed - including without distance data")
                # Still include but without distance data
                results.append({
                    'id': donation['id'],
                    'food_name': donation['food_name'],
                    'distance_km': None,
                    'travel_time_min': None,
                    'urgency_score': calculate_urgency_score(donation['expiry_datetime']),
                    'priority_score': None,
                    'is_feasible': None,
                    'time_until_expiry_min': None,
                    'donation': donation
                })
                continue
        
        # Calculate distance
        distance = calculate_distance(
            recipient_location['lat'],
            recipient_location['lng'],
            donor_lat,
            donor_lng
        )
        
        # Calculate urgency score
        urgency_score = calculate_urgency_score(donation['expiry_datetime'])
        
        # Calculate feasibility
        feasibility = calculate_feasibility(distance, donation['expiry_datetime'])
        
        # Calculate priority score based on strategy
        if strategy == 'distance':
            # Distance-first: Closer = higher priority (inverse distance)
            # Range: ~400 for very close (0.1km) to ~10 for far (40km)
            priority_score = (1 / (distance + 0.1)) * 40
            
        elif strategy == 'urgency':
            # Urgency-first: More urgent = higher priority
            # Range: 0-100 based on time until expiry
            priority_score = urgency_score
            
        else:  # balanced (default)
            # Balanced: Distance (40%) + Urgency (60%)
            distance_score = (1 / (distance + 0.1)) * 40
            priority_score = (urgency_score * 0.6) + (distance_score * 0.4)
        
        results.append({
            'id': donation['id'],
            'food_name': donation['food_name'],
            'food_type': donation['food_type'],
            'quantity': f"{donation['quantity']} {donation['unit']}",
            'pickup_city': donation['pickup_city'],
            'dietary_type': donation.get('dietary_type'),
            'spice_level': donation.get('spice_level'),
            'distance_km': round(distance, 2),
            'travel_time_min': feasibility['travel_time_minutes'],
            'time_until_expiry_min': feasibility['time_until_expiry_minutes'],
            'urgency_score': urgency_score,
            'priority_score': round(priority_score, 2),
            'is_feasible': feasibility['is_feasible'],
            'donor_lat': donor_lat,
            'donor_lng': donor_lng,
            'donation': donation
        })
    
    # Convert to DataFrame
    df = pd.DataFrame(results)
    
    if len(df) == 0:
        return df
    
    # Sort based on strategy
    df_with_priority = df[df['priority_score'].notna()].copy()
    df_without_priority = df[df['priority_score'].isna()].copy()
    
    if len(df_with_priority) > 0:
        # Primary sort by priority score (descending)
        df_with_priority = df_with_priority.sort_values('priority_score', ascending=False)
        
        # Add secondary sort for tie-breaking
        if strategy == 'distance':
            # If equal priority, prefer closer
            df_with_priority = df_with_priority.sort_values(
                ['priority_score', 'distance_km'], 
                ascending=[False, True]
            )
        elif strategy == 'urgency':
            # If equal priority, prefer more urgent
            df_with_priority = df_with_priority.sort_values(
                ['priority_score', 'time_until_expiry_min'], 
                ascending=[False, True]
            )
        else:  # balanced
            # If equal priority, prefer feasible ones first, then closer
            df_with_priority = df_with_priority.sort_values(
                ['priority_score', 'is_feasible', 'distance_km'],
                ascending=[False, False, True]
            )
    
    # Combine: items with priority first, then items without
    if len(df_without_priority) > 0:
        df_sorted = pd.concat([df_with_priority, df_without_priority], ignore_index=True)
    else:
        df_sorted = df_with_priority.reset_index(drop=True)
    
    return df_sorted

print("✅ Optimization function defined with multi-strategy support!")

## 5. Run Optimization and Display Results

In [ ]:
# Run the optimization with selected strategy
if len(donations) > 0:
    ranked_donations = find_optimal_donations(RECIPIENT_LOCATION, donations, strategy=SORTING_STRATEGY)
    
    print("\n" + "="*80)
    print(f"🏆 OPTIMIZATION RESULTS - {SORTING_STRATEGY.upper()} STRATEGY")
    print("="*80)
    
    # Display all ranked donations
    display_columns = [
        'food_name', 'food_type', 'quantity', 'pickup_city',
        'distance_km', 'travel_time_min', 'time_until_expiry_min',
        'urgency_score', 'priority_score', 'is_feasible'
    ]
    
    print("\n📊 All Donations Ranked by Priority:")
    if len(ranked_donations) > 0:
        display(ranked_donations[display_columns])
    
    # Show only feasible donations
    feasible = ranked_donations[ranked_donations['is_feasible'] == True] if len(ranked_donations) > 0 else pd.DataFrame()
    print(f"\n✅ FEASIBLE DONATIONS: {len(feasible)}")
    
    if len(feasible) > 0:
        print(f"\n🎯 Top 3 Recommended Donations ({SORTING_STRATEGY.upper()} strategy):")
        print("="*80)
        
        for idx in range(min(3, len(feasible))):
            row = feasible.iloc[idx]
            print(f"\n#{idx + 1}. {row['food_name']}")
            print(f"   Type: {row['food_type']}")
            print(f"   Quantity: {row['quantity']}")
            print(f"   Location: {row['pickup_city']}")
            print(f"   📍 Distance: {row['distance_km']} km")
            print(f"   🚗 Travel Time: {row['travel_time_min']} minutes (@ 20 km/h)")
            print(f"   ⏰ Time Until Expiry: {row['time_until_expiry_min']} minutes")
            print(f"   📊 Urgency Score: {row['urgency_score']}")
            print(f"   ⭐ Priority Score: {row['priority_score']}")
            print(f"   ✅ Deliverable: YES")
            print("-"*80)
    
    # Show infeasible donations
    infeasible = ranked_donations[ranked_donations['is_feasible'] == False] if len(ranked_donations) > 0 else pd.DataFrame()
    if len(infeasible) > 0:
        print(f"\n⚠️ INFEASIBLE DONATIONS (cannot reach before expiry): {len(infeasible)}")
        display(infeasible[['food_name', 'distance_km', 'travel_time_min', 'time_until_expiry_min']])
    
    # Show donations without location data
    no_location = ranked_donations[ranked_donations['distance_km'].isna()] if len(ranked_donations) > 0 else pd.DataFrame()
    if len(no_location) > 0:
        print(f"\n📍 DONATIONS WITHOUT LOCATION DATA: {len(no_location)}")
        print("   (These need GPS coordinates added to enable distance calculation)")
        display(no_location[['food_name', 'food_type', 'quantity', 'pickup_city']])
    
else:
    print("❌ No donations available to optimize")

## 6. Visualize Distance Distribution

In [ ]:
import matplotlib.pyplot as plt

if len(donations) > 0 and 'distance_km' in ranked_donations.columns:
    # Filter out null distances
    with_distance = ranked_donations[ranked_donations['distance_km'].notna()]
    
    if len(with_distance) > 0:
        # Create visualization
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
        
        # Distance histogram
        ax1.hist(with_distance['distance_km'], bins=10, color='#3b82f6', edgecolor='black', alpha=0.7)
        ax1.set_xlabel('Distance (km)', fontsize=12)
        ax1.set_ylabel('Number of Donations', fontsize=12)
        ax1.set_title('Distance Distribution', fontsize=14, fontweight='bold')
        ax1.grid(axis='y', alpha=0.3)
        
        # Priority score vs distance scatter
        colors = ['green' if x else 'red' for x in with_distance['is_feasible']]
        ax2.scatter(with_distance['distance_km'], with_distance['priority_score'], 
                   c=colors, alpha=0.6, s=100, edgecolors='black')
        ax2.set_xlabel('Distance (km)', fontsize=12)
        ax2.set_ylabel('Priority Score', fontsize=12)
        ax2.set_title('Priority Score vs Distance', fontsize=14, fontweight='bold')
        ax2.grid(True, alpha=0.3)
        ax2.legend(['Feasible', 'Infeasible'], loc='best')
        
        plt.tight_layout()
        plt.show()
        
        print("\n📈 Visualization complete!")
    else:
        print("⚠️ No donations with distance data to visualize")
else:
    print("⚠️ No data available for visualization")

## 7. Export Results to CSV (Optional)

In [ ]:
# Export ranked donations to CSV
if len(donations) > 0:
    output_file = 'ranked_donations.csv'
    
    export_columns = [
        'id', 'food_name', 'food_type', 'quantity', 'dietary_type', 'spice_level',
        'pickup_city', 'distance_km', 'travel_time_min', 'time_until_expiry_min',
        'priority_score', 'urgency_score', 'is_feasible'
    ]
    
    ranked_donations[export_columns].to_csv(output_file, index=False)
    print(f"✅ Results exported to: {output_file}")
    print(f"   Total rows: {len(ranked_donations)}")
else:
    print("⚠️ No data to export")

## 8. Summary Statistics

In [ ]:
if len(donations) > 0:
    with_distance = ranked_donations[ranked_donations['distance_km'].notna()]
    
    print("📊 SUMMARY STATISTICS")
    print("="*60)
    print(f"Total Donations: {len(ranked_donations)}")
    print(f"With Location Data: {len(with_distance)}")
    print(f"Without Location Data: {len(ranked_donations) - len(with_distance)}")
    
    if len(with_distance) > 0:
        print(f"\nFeasible Deliveries: {len(with_distance[with_distance['is_feasible'] == True])}")
        print(f"Infeasible Deliveries: {len(with_distance[with_distance['is_feasible'] == False])}")
        
        print(f"\nDistance Statistics:")
        print(f"  Min Distance: {with_distance['distance_km'].min():.2f} km")
        print(f"  Max Distance: {with_distance['distance_km'].max():.2f} km")
        print(f"  Avg Distance: {with_distance['distance_km'].mean():.2f} km")
        print(f"  Median Distance: {with_distance['distance_km'].median():.2f} km")
        
        print(f"\nTravel Time Statistics (@ 20 km/h):")
        print(f"  Min Travel Time: {with_distance['travel_time_min'].min()} min")
        print(f"  Max Travel Time: {with_distance['travel_time_min'].max()} min")
        print(f"  Avg Travel Time: {with_distance['travel_time_min'].mean():.0f} min")
        
        print(f"\nPriority Score Statistics:")
        print(f"  Highest Priority: {with_distance['priority_score'].max():.2f}")
        print(f"  Lowest Priority: {with_distance['priority_score'].min():.2f}")
        print(f"  Avg Priority: {with_distance['priority_score'].mean():.2f}")
    
    print("\n" + "="*60)
else:
    print("⚠️ No donations to analyze")